<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/157_LangGraph_Pricing_Compliance_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LangChain **state-style orchestrator**

Because you already structured your LangChain version using `node(state) -> state` functions, this becomes a near drop-in conversion. LangGraph's orchestration handles:

* ✅ **State propagation**
* ✅ **Node execution**
* ✅ **Conditional routing**
* ✅ **Workflow visualization/debugging**

---
## 💡 Benefits of This Version

| LangGraph Feature  | How It’s Used Here                                          |
| ------------------ | ----------------------------------------------------------- |
| ✅ State management | `AgentState` passed and mutated through all steps           |
| ✅ Modular nodes    | Each step is reusable and testable                          |
| ✅ Routing          | `route_decision()` replaces messy `if/else` blocks          |
| ✅ Observability    | Print summaries at each terminal node                       |
| ✅ Extensibility    | You can add human-in-loop, audit logs, retries, etc. easily |



In [ ]:
# ✨ Install LangGraph first (if needed)
# !pip install langgraph openai langchain faiss-cpu

# ================================
# 📦 Imports
from langgraph.graph import StateGraph, END
from langgraph.pregel import PregelState
from typing import TypedDict
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# ===========📄 Define Shared State
class AgentState(TypedDict, total=False):
    pricing: str
    rules: str
    summary: str
    risk: str

# ===========🧱 Node 1: Fetch Pricing Info
def fetch_node(state: AgentState) -> AgentState:
    state["pricing"] = "Product ID: 123\nBase Price: $100\nDiscount: 40%"
    return state

# ============📚 Node 2: RAG Compliance Policy Retrieval
def compliance_node(state: AgentState) -> AgentState:
    compliance_docs = [
        "Discounts over 30% require manager approval.",
        "All pricing must be auditable and linked to policy ID."
    ]
    embedding_model = OpenAIEmbeddings()
    vectorstore = FAISS.from_texts(compliance_docs, embedding_model)
    retriever = vectorstore.as_retriever()

    docs = retriever.get_relevant_documents(state["pricing"])
    state["rules"] = "\n".join([doc.page_content for doc in docs])
    return state

# ============🧠 Node 3: Evaluate Compliance via LLM
def evaluate_node(state: AgentState) -> AgentState:
    llm = OpenAI(temperature=0)
    prompt = PromptTemplate.from_template("""
    You are a compliance agent. Given the product pricing and relevant rules,
    evaluate if the pricing complies with policy. Summarize findings clearly.

    Pricing Info:
    {pricing}

    Compliance Rules:
    {rules}
    """)
    chain = LLMChain(llm=llm, prompt=prompt)
    state["summary"] = chain.run({"pricing": state["pricing"], "rules": state["rules"]})

    # Tag with risk level
    if "require manager approval" in state["summary"].lower():
        state["risk"] = "high"
    else:
        state["risk"] = "low"
    return state

# ==========✅ Node 4A: Approved Output
def approve_node(state: AgentState) -> AgentState:
    print("✅ Pricing Approved.")
    print("📝 Summary:", state["summary"])
    return state

# ==========🚨 Node 4B: Escalation
def escalate_node(state: AgentState) -> AgentState:
    print("🚨 Escalation Triggered: Policy Violation Detected.")
    print("📝 Summary:", state["summary"])
    return state

# ==========🔁 Routing Function (Decision Logic)
def route_decision(state: AgentState) -> str:
    return "escalate" if state["risk"] == "high" else "approve"

# ===========🧠 Build LangGraph Flow
builder = StateGraph()

# Add steps
builder.add_node("fetch", fetch_node)
builder.add_node("compliance", compliance_node)
builder.add_node("evaluate", evaluate_node)
builder.add_node("approve", approve_node)
builder.add_node("escalate", escalate_node)

# Add connections
builder.add_edge("fetch", "compliance")
builder.add_edge("compliance", "evaluate")
builder.add_conditional_edges("evaluate", route_decision, {
    "approve": "approve",
    "escalate": "escalate"
})
builder.add_edge("approve", END)
builder.add_edge("escalate", END)

# Compile the graph
graph = builder.compile()

# ==========🚀 Run the Agent
initial_state: AgentState = {}
graph.invoke(initial_state)



## 1. ✅ **State Propagation**

> **“A single shared state dictionary moves through each node and is updated along the way.”**

### What it means:

* Each node (like `fetch_node`, `evaluate_node`) receives a copy of the current `state`.
* It **reads from it**, **modifies it**, and **returns it**.
* LangGraph takes care of **thread-safe state propagation** between nodes.

### In your code:

```python
def fetch_node(state: AgentState) -> AgentState:
    state["pricing"] = "Product ID: 123\nBase Price: $100\nDiscount: 40%"
    return state
```

* This function updates the shared state.
* You **don’t need to manually pass arguments between steps** — LangGraph handles it.

### Why it matters:

* Encourages **modular, functional design** (each node only touches its part of state).
* Avoids tight coupling — any node can be reused elsewhere.
* Makes the workflow auditable and inspectable (like logs or DAGs in Airflow).

---

## 2. ✅ **Node Execution**

> **“Each step in the graph is a named node — LangGraph calls and manages them based on edges and state.”**

### What it means:

* You define each step as a **node** with a name:

```python
builder.add_node("fetch", fetch_node)
builder.add_node("evaluate", evaluate_node)
```

* Nodes can run anything: LLM calls, API fetches, data pipelines, tools, etc.
* You focus on **what each node does**, not on control logic.

### Why it matters:

* Decouples logic from execution flow.
* You can **debug or log per-node**.
* Can be extended to include **timeouts, retries, human handoff, async**, etc.

---

## 3. ✅ **Conditional Routing**

> **“You can route the flow based on values in state — like `if` logic, but declarative.”**

### In your code:

```python
def route_decision(state: AgentState) -> str:
    return "escalate" if state["risk"] == "high" else "approve"

builder.add_conditional_edges("evaluate", route_decision, {
    "approve": "approve",
    "escalate": "escalate"
})
```

* This replaces `if/else` logic and embeds it in the **graph definition**.
* LangGraph uses the returned key (`"approve"` or `"escalate"`) to decide what edge to follow.

### Why it matters:

* Makes your logic **explicit, visual, and testable**.
* Easy to swap in new routing logic (e.g., from an LLM instead of hard rules).
* Encourages **auditable decision paths** (e.g., for compliance).

---

## 4. ✅ **Workflow Visualization & Debugging**

> **“LangGraph supports runtime inspection and graph visualization — not just logic, but flow.”**

### What it means:

* You can visualize your DAG (directed acyclic graph) or flow:

```python
graph.get_graph().draw_mermaid()
```

* Each node becomes a **visual block**, each edge a **conditional path**.
* You can **log intermediate state** at each step to a file, UI, or dashboard.

### Why it matters:

* Helps **debug agent behavior** without reading logs line-by-line.
* Makes it easier to **explain** or **audit** the workflow.
* You can add:

  * **Retries**
  * **Timeouts**
  * **Human-in-the-loop stops**
  * **Subgraphs or parallel paths**

---

## 🧠 TL;DR: LangGraph’s DNA = Workflow Thinking

| Feature   | What You Do                 | LangGraph Handles               |
| --------- | --------------------------- | ------------------------------- |
| Logic     | Write clean node functions  | Executes them in order          |
| State     | Update shared `state` dict  | Passes state to each node       |
| Branching | Return `"next_step"` string | Routes to the correct node      |
| Flow      | Define edges + decisions    | Visualizes & controls execution |



Thanks for sharing your **Agent Recipe** — it's a strong architectural guide for designing intelligent, testable, and maintainable agent systems. Now that we’ve looked at all four frameworks (LangChain, LangGraph, CrewAI, AutoGen), and compared them to your principles, I’ll answer your key question:

> **Which framework best follows my agent design principles?**

---

## 🧭 Your Agent Design Principles

| Principle                               | Description                                                                     |
| --------------------------------------- | ------------------------------------------------------------------------------- |
| ✅ **Modular, functional design**        | Each component (tool, node, step) does one thing and does it well.              |
| ✅ **Loose coupling**                    | Tools/components don’t have hidden dependencies and are easy to test and reuse. |
| ✅ **Auditable & inspectable workflows** | You can trace what happened: logs, state, DAGs, transcripts, etc.               |

---

## 🧪 Comparison: Which Framework Best Fits?

| Framework                      | Modular/Functional                                 | Loose Coupling                                                       | Auditable/Inspectable                                     | Overall Alignment        |
| ------------------------------ | -------------------------------------------------- | -------------------------------------------------------------------- | --------------------------------------------------------- | ------------------------ |
| **✅ LangGraph**                | ✅ Excellent — nodes encapsulate one function       | ✅ State is explicit and scoped                                       | ✅ Full DAGs, visual workflows, state history              | 🟩 **Highest** alignment |
| **CrewAI**                     | ✅ Agents and tasks are modular                     | ⚠️ Medium — tools/tasks often assume shared state or agent knowledge | ✅ Good logs, task traces                                  | 🟨 Medium                |
| **LangChain (Tools + Agents)** | ⚠️ Mixed — tools are modular, chains can get messy | ⚠️ Can become tightly coupled due to prompt chaining, memory         | ✅ Execution traces and intermediate steps are inspectable | 🟨 Medium                |
| **AutoGen**                    | ✅ Agents and messages are modular                  | ⚠️ Moderate — code interleaves controller logic with prompts/tools   | ⚠️ Less formal logging unless customized                  | 🟨 Medium–low            |

---

## 🥇 Winner: **LangGraph**

LangGraph most closely mirrors your **agent recipe**:

* **Nodes** are cleanly separated, each touching only a slice of the state — just like your **Tool** classes.
* **State is explicit**, passed forward in every step — mirrors your `ActionContext` and memory.
* **Visual auditability** through the DAG structure — just like your emphasis on observability.
* **Composable + testable** — you can dry-run one node, swap it, reuse it — same as your modular capabilities in your framework.

In contrast:

* **LangChain** and **AutoGen** require more discipline to enforce clean separation of concerns.
* **CrewAI** tries to be more declarative, but that sometimes leads to **black-box behavior** or tight coupling across tasks.

---

## ✅ Recommendation

Since you're **already thinking in LangGraph-style** (stateful, modular, auditable), it makes sense to:

* **Double down on LangGraph** — it natively supports your architectural values.
* **Use your Agent Recipe as a blueprint** for every LangGraph project (especially in enterprise contexts).
* Treat **CrewAI and AutoGen** as complementary — explore them for when coordination or autonomous dialog between agents is critical.

Would you like me to translate your full agent recipe into a reusable LangGraph project scaffold next? That way you can start with your exact structure but powered by LangGraph.


We’ll now build a **LangGraph scaffold** based on your **Agent Recipe**, using **placeholder nodes, dummy state**, and comments to mirror your architecture. Think of this like a "blank agent skeleton" that you can reuse across projects.

---

## 🧠 Goal

> Create a reusable **LangGraph project scaffold** that mirrors your agent recipe:

* Modular, functional nodes
* Clean state handling
* Auditable workflow
* Easy to plug in tools, memory, and LLMs later

---

## 📁 Structure Overview

```
project/
│
├── main.py                  ← Builds and runs the graph
├── nodes/
│   ├── fetch_node.py        ← Tool node: gets external data
│   ├── policy_node.py       ← LLM + RAG node
│   ├── eval_node.py         ← Decision/evaluator node
│   └── escalate_node.py     ← Optional human loop / manager node
│
├── state.py                 ← AgentState definition
└── graph_builder.py         ← Graph construction logic
```

---

## 🧩 1. `state.py` – Define Shared AgentState

```python
from langgraph.graph import State

class AgentState(State):
    pricing: str = None
    policies: str = None
    decision: str = None
    escalation_required: bool = False
```

---

## 🔧 2. `nodes/fetch_node.py` – Fetch Input

```python
from state import AgentState

def fetch_node(state: AgentState) -> AgentState:
    # 🚧 TODO: Replace with real tool/DB call
    state["pricing"] = "Dummy pricing data"
    return state
```

---

## 📚 3. `nodes/policy_node.py` – Retrieve Rules via RAG

```python
from state import AgentState

def policy_node(state: AgentState) -> AgentState:
    # 🚧 TODO: Call RAG chain here
    state["policies"] = "Dummy policies related to pricing"
    return state
```

---

## ⚖️ 4. `nodes/eval_node.py` – Decision Logic

```python
from state import AgentState

def eval_node(state: AgentState) -> AgentState:
    # 🚧 TODO: Add rule evaluation or LLM reasoning
    state["decision"] = "approve" if "40%" in state["pricing"] else "escalate"
    state["escalation_required"] = state["decision"] == "escalate"
    return state
```

---

## 📤 5. `nodes/escalate_node.py` – Optional Human Escalation

```python
from state import AgentState

def escalate_node(state: AgentState) -> AgentState:
    # 🚧 TODO: Send alert or ask for manager approval
    return state
```

---

## 🧱 6. `graph_builder.py` – Graph Construction

```python
from langgraph.graph import StateGraph, END
from state import AgentState
from nodes.fetch_node import fetch_node
from nodes.policy_node import policy_node
from nodes.eval_node import eval_node
from nodes.escalate_node import escalate_node

def build_graph():
    builder = StateGraph(AgentState)

    builder.add_node("fetch", fetch_node)
    builder.add_node("policy", policy_node)
    builder.add_node("evaluate", eval_node)
    builder.add_node("escalate", escalate_node)

    builder.set_entry_point("fetch")

    builder.add_edge("fetch", "policy")
    builder.add_edge("policy", "evaluate")

    def router(state: AgentState) -> str:
        return "escalate" if state["escalation_required"] else END

    builder.add_conditional_edges("evaluate", router, {
        "escalate": "escalate",
        END: END
    })

    return builder.compile()
```

---

## 🚀 7. `main.py` – Run the Graph

```python
from graph_builder import build_graph
from state import AgentState

if __name__ == "__main__":
    graph = build_graph()

    initial_state = AgentState()
    final_state = graph.invoke(initial_state)

    print("🧾 Final Decision:", final_state["decision"])
```

---

## ✅ What You Can Do Next

| Goal           | How                                                          |
| -------------- | ------------------------------------------------------------ |
| Add real tools | Replace `fetch_node` with LangChain Tool/agent/toolkit       |
| Add RAG        | Swap in LangChain Retriever or GraphRAG inside `policy_node` |
| Log flow       | Print state at each node, or integrate tracing               |
| Visualize      | `graph.get_graph().draw_mermaid()` for DAG                   |




Structuring your LangGraph project with **modular, node-specific scripts** like:

```
nodes/
├── fetch_node.py
├── policy_node.py
├── eval_node.py
├── escalate_node.py
```

…is not just cleaner — it's also one of the **best architectural moves** you can make when building scalable and maintainable agent workflows.

Here’s **why this structure is such a win**:

---

## ✅ Why Splitting Nodes into Separate Files Is Great

### 1. 🧠 **Cognitive clarity**

Each file has *one purpose*. You don’t need to scroll or context-switch between unrelated logic — just open the file you care about.

> “I want to debug the policy retriever → open `policy_node.py`.”

---

### 2. 🔄 **Reusability**

Each node can become a **mini building block** — reusable across different flows.

* Example: `fetch_node.py` could work in both pricing and inventory agents.
* Swappable like LEGO bricks.

---

### 3. 🧪 **Unit testing is easy**

You can write a test for **just** the node:

```python
def test_policy_node():
    state = AgentState(pricing="Product 123 with 50% discount")
    result = policy_node(state)
    assert "Discounts over 30%" in result["policies"]
```

No need to spin up the full LangGraph — test one file, one function.

---

### 4. 📁 **Clean separation of concerns**

* `graph_builder.py` knows *how* the nodes are connected.
* `nodes/*.py` knows *what* each step does.
* No tangled logic, no monolithic spaghetti.

---

### 5. 📈 **Scaling to teams**

If you're working on a team:

* Each person can own a node or group of nodes.
* Easy to review, version, document, and evolve independently.

---

### 6. 🛠️ **Plug-and-play with LangChain, tools, APIs**

If you adopt this pattern:

* You can experiment with different RAG chains in `policy_node.py`
* Try different LLM models in `eval_node.py`
* Or swap `fetch_node.py` from static mock data → live API fetch → database query → LangChain tool → all without touching other logic.

---

## 🧠 TL;DR

Yes, this is a **highly recommended structure** — and it aligns perfectly with your **Agent Recipe** principles:

| Agent Recipe Principle | This Structure Delivers                 |
| ---------------------- | --------------------------------------- |
| ✅ Modular              | Each node = one function = one file     |
| ✅ Loosely Coupled      | Nodes interact only via `state`         |
| ✅ Auditable            | You can log/test each node in isolation |






> **`AgentState` is the central nervous system of a LangGraph agent.**
> It carries all the knowledge, decisions, results, and metadata from one node to the next — like a **living memory stream**.

Let’s break it down clearly:

---

## 🧠 What Is `AgentState` in LangGraph?

In LangGraph, the agent doesn't pass variables around like functions.
Instead, all nodes **read from and write to a shared dictionary-like object** called `AgentState`.

It’s the **"single source of truth"** for:

* Inputs (user query, retrieved data)
* Outputs (decisions, approvals, actions)
* Control logic (flags, booleans)
* Memory (conversation history, tool traces)

---

## 🧬 Anatomy of an AgentState

Here’s a minimal example:

```python
from langgraph.graph import State

class AgentState(State):
    pricing: str = None
    policies: str = None
    decision: str = None
    escalation_required: bool = False
```

This defines a **schema** for your state:

* It's like a strongly-typed dictionary (`Dict[str, Any]` with structure).
* You can treat it like a Python dict:

  ```python
  state["pricing"] = "Product ID 123, 50% discount"
  ```

---

## 🔁 How `AgentState` Flows Through Nodes

```mermaid
graph TD
    Start --> FetchNode --> PolicyNode --> EvalNode --> Decision

    subgraph state
      S1[pricing]
      S2[policies]
      S3[decision]
      S4[escalation_required]
    end
```

* At **each node**, you mutate the shared `state` object.
* When the node returns, the **updated state is passed** to the next node.
* This allows **pure functions**: input → process → output via state.

---

## ✨ Benefits of `AgentState`

| Feature                     | Why it’s useful                                         |
| --------------------------- | ------------------------------------------------------- |
| ✅ Centralized memory        | All agent knowledge lives in one consistent place       |
| ✅ Decouples nodes           | Nodes don’t need to know each other’s inputs            |
| ✅ Easy to debug             | You can print/inspect `state` at any node               |
| ✅ Enables conditional logic | Just read flags from `state`                            |
| ✅ Schema-safe               | If you subclass `State`, it behaves like a typed object |

---

## 🧪 State Debugging Example

At the end of your graph, you can inspect everything:

```python
result = graph.invoke(initial_state)

for k, v in result.items():
    print(f"{k}: {v}")
```

Gives you a full trace like:

```
pricing: Product 123, discount 40%
policies: Max allowed discount is 30%
decision: escalate
escalation_required: True
```

---

## 🧱 Advanced State Usage

You can go deeper with:

* **List or dict fields**:

  ```python
  messages: List[dict]
  tools_used: List[str]
  ```

* **Embedding history**:

  ```python
  memory: ConversationBufferMemory
  ```

* **Versioning or audit**:

  ```python
  steps_taken: List[str]
  ```

* **LLM chaining logic**:

  * Store intermediate generations
  * Track how state evolved at each node

---

## 🧠 TL;DR

| Concept       | Meaning                               |
| ------------- | ------------------------------------- |
| `AgentState`  | Your agent's working memory           |
| Acts like     | A dict with structure                 |
| Moves through | Every node                            |
| You can       | Read, write, mutate                   |
| Benefits      | Modularity, auditability, testability |



One of the **nicest features** of LangGraph is that the **entire `AgentState` is inspectable like a dictionary**, so you can easily audit everything your agent has done.

Here’s a **simple example**:

```python
result = graph.invoke(initial_state)

for k, v in result.items():
    print(f"{k}: {v}")
```

Output:

```
pricing: Product 123, 40% discount
policies: Discounts over 30% require VP approval
decision: escalate
escalation_required: True
```

---

## 🔎 Why This Is So Useful

| Feature        | Benefit                                                            |
| -------------- | ------------------------------------------------------------------ |
| ✅ Transparent  | You see *everything* the agent saw, decided, and stored            |
| ✅ Debuggable   | Spot bugs instantly: missing input, bad policy match, faulty logic |
| ✅ Auditable    | Trace every step of the decision, like a black box flight recorder |
| ✅ Reproducible | You can replay a state through any node for testing                |

---

## 🛠️ Advanced Tip

You can even **capture and inspect intermediate state** at each node by adding logs or prints inside them:

```python
def eval_node(state: AgentState) -> AgentState:
    print("🔍 Evaluating decision with state:", state)
    ...
    return state
```

Or if you want **state snapshots at every node**, you can wrap your nodes like this:

```python
def trace_wrapper(fn):
    def wrapped(state):
        print(f"📍 {fn.__name__} START")
        new_state = fn(state)
        print(f"📍 {fn.__name__} OUTPUT:", new_state)
        return new_state
    return wrapped

# Then register nodes like:
builder.add_node("evaluate", trace_wrapper(eval_node))
```


